## Greedy Sentiment Transformation
11/13/16 - Use a greedy switching method to exchange words based on best antonym.
Uses the IMDB dataset folder (http://ai.stanford.edu/~amaas/data/sentiment/).

In [16]:
from sentiment_utils import *
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
from nltk.corpus import wordnet as wn

In [35]:
def greedy_transform_func(filename, review, score):
    sentiment_analyzer = SentimentIntensityAnalyzer()
    tagged_review = nltk.pos_tag(word_tokenize(review))
    transformed_review = []
    for tagged_word in tagged_review:
        if tagged_word[1] in ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']:
            #1) get sentiment of the tagged word
            #2) if the sentiment of the tagged word is opposite of the review score
            #3) -get the antonyms of tagged word
            #4) -for each antonym, score sentiment, or pick random sentiment?
            #5) -append antonym
            word_sentiment = sentiment_analyzer.polarity_scores(tagged_word[0])['compound']
            print tagged_word[0], tagged_word[1], word_sentiment, score
            if word_sentiment*(score - 5) > 0:
                antonyms = get_antonyms(tagged_word[0], tagged_word[1])
                if len(antonyms) == 0:
                    transformed_review.append('not ' + tagged_word[0])
                else: transformed_review.append(antonyms[0])
            else: transformed_review.append(tagged_word[0])
        else:
            transformed_review.append(tagged_word[0])
    return " ".join(transformed_review)

def get_antonyms(word, word_pos):
    all_antonyms = []
    pos_dict = {'JJ': wn.ADJ, 'JJR': wn.ADJ, 'JJS': wn.ADJ, 'RB': wn.ADV, 'RBR': wn.ADV, 'RBS': wn.ADV}
    wn_pos = pos_dict[word_pos]
    for syn in wn.synsets(word, pos = pos_dict[word_pos]):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                all_antonyms.append(lemma.antonyms()[0].name())
    antonyms = list(set(all_antonyms))
    return antonyms

Greedy Test One Review

In [36]:
for (filename, review, score) in imdb_sentiment_reader(dataset_type='val', sentiment='pos'):
    print "Original review: "
    print review
    print "Transformed review:" 
    transformed = greedy_transform_func(filename, review, score)
    print transformed
    break

Original review: 
I went and saw this movie last night after being coaxed to by a few friends of mine. I'll admit that I was reluctant to see it because from what I knew of Ashton Kutcher he was only able to do comedy. I was wrong. Kutcher played the character of Jake Fischer very well, and Kevin Costner played Ben Randall with such professionalism. The sign of a good movie is that it can toy with our emotions. This one did exactly that. The entire theater (which was sold out) was overcome by laughter during the first half of the movie, and were moved to tears during the second half. While exiting the theater I not only saw many women in tears, but many full grown men as well, trying desperately not to let anyone see them crying. This movie was great, and I suggest that you go see it before you judge.
Transformed review:
last JJ 0.0 10
few JJ 0.0 10
reluctant JJ -0.25 10
only RB 0.0 10
able JJ 0.0 10
wrong JJ -0.4767 10
very RB 0.0 10
well RB 0.2732 10
such JJ 0.0 10
good JJ 0.4404 10
